In [ ]:
import openai
import pandas as pd
import os

In [ ]:
# openai.api_key = 

In [ ]:
MODEL = "gpt-4"

In [ ]:
from tenacity import retry, stop_after_attempt, wait_chain, wait_fixed

In [ ]:
@retry(wait=wait_chain(*[wait_fixed(3) for i in range(3)] +
                       [wait_fixed(5) for i in range(2)] +
                       [wait_fixed(10)]))
def completion_with_backoff(**kwargs):
    return openai.ChatCompletion.create(**kwargs)

In [ ]:
def format_prompt(word):
    instruction = '''A word contains negation if it has a negated meaning, usually expressed through a negative prefix (such as un, in) or suffix (such as less).\n\nThe word unwell contains negation. True or False?\nAnswer: True\nExplanation: unwell is created by prepending the root word well with the negative prefix un.\n\nThe word uniform contains negation. True or False?\nAnswer: False\nExplanation: uniform just coincidentally starts with un.\n\nBe concise and only answer with the given options.\n\n'''
    prompt = "The word "+ word + " contains negation."+ " True or False?\nAnswer:"
    message = instruction + prompt
    message = prompt
    
    messages = [
        {"role": "system", "content": 'Be precise and only answer with the given options.'},
        {"role": "user", "content": message},
    ]
    return messages

In [ ]:
def request(samples):
    ret = []
    for sample in tqdm(samples):
        messages = format_prompt(sample)
        response = completion_with_backoff(
            model=MODEL,
            messages= messages,
            temperature=0,
        )
        ans_model = response['choices'][0]['message']['content']
        # print(messages)
        print(sample, ans_model)
        ret.append([sample, ans_model])
        # print(ans_model)
        # print('===================')
    return ret

In [ ]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset('joey234/affixal_negation')

In [ ]:
words = dataset['train']['word']

In [ ]:
neg_words = []
labels = []

In [ ]:
for i, word in enumerate(words):
    if dataset['train']['hyphen'][i] == '-':
        continue
    else:
        words[i] = dataset['train']['hyphen'][i]
        neg_words.append(dataset['train']['hyphen'][i])
        labels.append(dataset['train']['label'][i])

In [ ]:
neg_words[:10]

In [ ]:
ret_neg = request(neg_words)

In [ ]:
len(words)

In [ ]:
df_gpt4_pred_hyphen_neg = pd.DataFrame( data = {'word': neg_words, 'pred': [x[1] for x in ret_neg], 'gold': labels})

In [ ]:
df_gpt4_pred_hyphen_neg['pred'] = df_gpt4_pred_hyphen_neg['pred'].map(lambda x: labelmap[x])

In [ ]:
len(df_gpt4_pred_hyphen_neg[df_gpt4_pred_hyphen_neg['pred'] == df_gpt4_pred_hyphen_neg['gold']]) / len(df_gpt4_pred_hyphen_neg)

In [ ]:
ret_values = []
for i,pred in enumerate(ret):
    pred_value = pred[1].split('\n')[0]
    ret_values.append([pred[0], pred_value])

In [ ]:
len(ret_values)

In [ ]:
ret_values == ret

In [ ]:
df = pd.DataFrame(data = {'word': dataset['train']['word'], 'pred': [x[1] for x in ret_values], 'gold': dataset['train']['label']})

In [ ]:
df

In [ ]:
labelmap = {'False':1, 'True':0}

In [ ]:
rows = []
for i,row in df.iterrows():
    if row['pred'] not in ["True", "False"]:
        row['pred'] = "False"
    rows.append(row)
df = pd.DataFrame(data = rows)
# df[df['pred'] == "Without providing a specific word, it's impossible to determine whether this statement is true or false."]['pred'] = False

In [ ]:
df['pred'] = df['pred'].map(lambda x: labelmap[x])

In [ ]:
df.to_csv('gpt4_pred_hyphen.csv', index = None)

In [ ]:
df = pd.read_csv('gpt4_zero_pred.csv')

In [ ]:
df

In [ ]:
len(df[df['pred'] == df['gold']])/len(df)

In [ ]:
df_neg = df[df['gold'] == 0]
len(df_neg[df_neg['pred'] == df_neg['gold']])/len(df_neg)

In [ ]:
df_pos = df[df['gold'] == 1]
len(df_pos[df_pos['pred'] == df_pos['gold']])/len(df_pos)

In [ ]:
hyphens = dataset['train']['hyphen']

In [ ]:
hyphens = list(set(hyphens))

In [ ]:
len(hyphens)

In [ ]:
hyphens.remove('-')

In [ ]:
len(hyphens)

In [ ]:
ret_values = []
for i,pred in enumerate(ret):
    pred_value = pred[1].split('\n')[0]
    ret_values.append([pred[0], pred_value])

In [ ]:
len(ret_values)

In [ ]:
len(ret)

In [ ]:
correct = 0
for pred in ret_values:
    if pred[1] == 'True':
        correct+=1
print(correct/len(ret))

In [ ]:
df = pd.DataFrame(data = {'word': dataset['train']['word'], 'pred': [x[1] for x in ret], 'gold': dataset['train']['label']})

In [ ]:
df

In [ ]:
labelmap = {'False':1, 'True':0}

In [ ]:
df['pred'] = df['pred'].map(lambda x: labelmap[x])

In [ ]:
len(df[df['pred'] == df['gold']])/len(df)

In [ ]:
df.to_csv('gpt4_pred.csv', index = None)

In [ ]:
df = pd.read_csv('gpt4_pred.csv')

In [ ]:
len(df[df['pred'] == df['gold']])/len(df)

In [ ]:
df_neg = df[df['gold'] == 0]
len(df_neg[df_neg['pred'] == df_neg['gold']])/len(df_neg)

In [ ]:
df_pos = df[df['gold'] == 1]
len(df_pos[df_pos['pred'] == df_pos['gold']])/len(df_pos)

In [ ]:
df

In [ ]:
original_df = pd.read_csv('tokenized_data.csv')

In [ ]:
original_df

In [ ]:
gpt4_df = original_df[original_df['tokenizer']=='cl100k_base']
flan_df = original_df[original_df['tokenizer']=='google/flan-t5-xxl']
llama2_df = original_df[original_df['tokenizer']=='meta-llama/Llama-2-13b-chat-hf']


In [ ]:
gpt4_pred = pd.read_csv('gpt4_pred.csv')

In [ ]:
flan_pred = pd.read_csv('flant5_pred.csv')
llama2_pred = pd.read_csv('llama2_pred.csv')

In [ ]:
rows = []
for i,row in gpt4_df.iterrows():
    row['Predict'] = gpt4_pred.loc[gpt4_pred['word'] == row['neg_word']]['pred'].values[0]
    # print(row['neg_word'], row['Predict'])
    rows.append(row)

for i,row in flan_df.iterrows():
    row['Predict'] = flan_pred.loc[flan_pred['word'] == row['neg_word']]['pred'].values[0]
    # print(row['neg_word'], row['Predict'])
    rows.append(row)


for i,row in llama2_df.iterrows():
    row['Predict'] = llama2_pred.loc[llama2_pred['word'] == row['neg_word']]['pred'].values[0]
    # print(row['neg_word'], row['Predict'])
    rows.append(row)

In [ ]:
df_results = pd.DataFrame(data = rows)

In [ ]:
df_results

In [ ]:
df_results['soft_neg_correct'].unique()

In [ ]:
temps = []
for i,row in df_results.iterrows():
    if row['neg_correct'] == 'Not segmented':
        row['NegMorph'] = 'Under-segmented'
    elif row['neg_correct'] == 'Correct':
        row['NegMorph'] = 'Correct'
    elif row['soft_neg_correct'] == 'Incorrect':
        row['NegMorph'] = 'Over-segmented'
    elif row['soft_neg_correct'] == 'Correct' and row['neg_correct'] == 'Incorrect':
        row['NegMorph'] = 'Under-segmented'
    temps.append(row)
df_results = pd.DataFrame(data = temps)

In [ ]:
df_results['NegMorph'].unique()

In [ ]:
# Calculate the total count for each 'tokenizer_type'                                                                                                     
total_count = df_results['NegMorph'].value_counts()                                                                                                         
                                                                                                                                                        
# Calculate the count for each 'tokenizer_type' and 'correct' combination                                                                                 
correct_count = df_results[df_results['Predict'] == 0]['NegMorph'].value_counts()                                                                                
incorrect_count = df_results[df_results['Predict'] == 1]['NegMorph'].value_counts()                                                                             
                                                                                                     
# Calculate the percentage                                                                                                                                
percentage_correct = correct_count / total_count * 100                                                                                                    
percentage_incorrect = incorrect_count / total_count * 100                                                                                                
                                                                                                                                                        
percentage_correct, percentage_incorrect

In [ ]:
import matplotlib.pyplot as plt                                                                                                                           
import seaborn as sns                                                                                                                                     


In [ ]:
# df_results['NegMorph'] 
# df_error = df_results[df_results['Predict'] == 1]
# order = ['Correct', 'Over-segmented', 'Under-segmented']
g = sns.barplot(
    data=df_results,
    x="tokenizer", y="Predict" ,hue= 'NegMorph', palette = ['g', '#bcbd22', '#d62728'], alpha = 1, errorbar = None, width = 0.6
)
g.set_ylabel("Error Rate")
g.set_xlabel("")

g.set_xticklabels(["GPT-4", "Flan-T5-xxl", "Llama2-13B"])
# g.legend
g.legend_.set_title('NegMorph')
plt.tight_layout()

fig = g.get_figure()
sns.despine(fig)

fig.savefig("negmorph_dist.pdf") 

In [ ]:
df

In [ ]:
rows = []
for i, row in df_results.iterrows():
    if row['neg_correct'] == 'Not segmented':
        row['soft_neg_correct'] = 'Not segmented'
    rows.append(row)
df_results = pd.DataFrame(data = rows)

In [ ]:
temps = []
for i,row in df_results.iterrows():
    if row['soft_neg_correct'] == 'Not segmented':
        row['NegMorph'] = 'Not segmented'
    elif row['neg_correct'] == 'Correct':
        row['NegMorph'] = 'Correct'
    elif row['soft_neg_correct'] == 'Incorrect':
        row['NegMorph'] = 'Over-segmented'
    elif row['soft_neg_correct'] == 'Correct' and row['neg_correct'] == 'Incorrect':
        row['NegMorph'] = 'Under-segmented'
    temps.append(row)

In [ ]:
df_results = pd.DataFrame(data = temps)

In [ ]:
df_error = df_results[df_results['Predict'] == 1]

In [ ]:
df_error

In [ ]:
df_results.to_csv('results.csv')

In [ ]:
import seaborn as sns

In [ ]:
order = ['Correct', 'Over-segmented', 'Under-segmented', 'Not segmented']

In [ ]:
g = sns.barplot(
    data=df_results,
    x="tokenizer", y="Predict" ,hue= 'NegMorph', palette = ['#1f77b4','orange', 'gold', '#d62728'], alpha = 1, errorbar = None,
    hue_order = order
)
g.set_ylabel("Error Rate")
g.set_xlabel("")

g.set_xticklabels(["GPT-4", "Flan-T5-xxl", "LlaMa2-13B"])
# g.legend
g.legend_.set_title('NegMorph')
fig = g.get_figure()
fig.savefig("negmorph_finegrained_dist.pdf") 


In [ ]:
from tqdm import tqdm

In [ ]:
def format_prompt_sentiment(word):
    # instruction = '''A word contains negation if it has a negated meaning, usually expressed through a negative prefix (such as un, in) or suffix (such as less).\n\nThe word unwell contains negation. True or False?\nAnswer: True\nExplanation: unwell is created by prepending the root word well with the negative prefix un.\n\nThe word uniform contains negation. True or False?\nAnswer: False\nExplanation: uniform just coincidentally starts with un.\n\nBe concise and only answer with the given options.\n\n'''
    fewshot_context = '''The sentiment of the word amazing is positive, negative, or neutral?\nAnswer: positive\n\nThe sentiment of the word bored is positive, negative, or neutral?\nAnswer: negative\n\nThe sentiment of the word intended is positive, negative, or neutral?\nAnswer: neutral\n\n'''
    prompt = fewshot_context + "The sentiment of the word "+ word + " is positive, negative, or neutral?\nAnswer:"
    # message = instruction + prompt
    message = prompt
    
    messages = [
        # {"role": "system", "content": 'Be precise and only answer with the given options.'},
        {"role": "user", "content": message},
    ]
    return messages

In [ ]:
def request_sentiment(samples):
    ret = []
    for sample in tqdm(samples):
        messages = format_prompt_sentiment(sample)
        response = completion_with_backoff(
            model=MODEL,
            messages= messages,
            temperature=0,
        )
        ans_model = response['choices'][0]['message']['content']
        # print(messages)
        # print(sample, ans_model)
        ret.append([sample, ans_model])
        # print(ans_model)
        # print('===================')
    return ret

In [ ]:
def format_prompt_sentiment_sentence(sentence):
    prompt =  sentence+"\nQuestion: Is this review positive or negative?\nAnswer:"
    message = prompt
    
    messages = [
        {"role": "system", "content": 'Be precise and only answer with the given options: Positive or Negative. In case it is unclear, answer with the more likely option.'},
        {"role": "user", "content": message},
    ]
    return messages

In [ ]:
def request_sentiment_sentence(samples):
    ret = []
    for sample in tqdm(samples):
        messages = format_prompt_sentiment_sentence(sample)
        response = completion_with_backoff(
            model=MODEL,
            messages= messages,
            temperature=0,
        )
        ans_model = response['choices'][0]['message']['content']
        print(messages)
        # print(sample, ans_model)
        ret.append([sample, ans_model])
        # print(ans_model)
        # print('===================')
    return ret

In [ ]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset('joey234/rotten_tomatoes_affix_neg')

In [ ]:
sentences = dataset['test']['text']

In [ ]:
len(sentences)

In [ ]:
ret = request_sentiment_sentence(sentences)

In [ ]:
for x in ret:
    if x[1] not in ['Positive','Negative']:
        print(x)

In [ ]:
gpt4_rt_affix = pd.DataFrame(data = {'word': dataset['test']['text'], 'pred': [x[1] for x in ret], 'gold': dataset['test']['label']})

In [ ]:
labelmap = {'Positive':1, 'Negative':0, 'Neutral':2 ,'Unclear':3}

In [ ]:
gpt4_rt_affix['pred'] = gpt4_rt_affix['pred'].map(lambda x: labelmap[x])

In [ ]:
gpt4_rt_affix.to_csv('gpt4_rt_affix.csv', index = False)

In [ ]:
dataset = load_dataset('joey234/rotten_tomatoes_affix_neg')
sentences = dataset['test']['sentence_replace_affix']
ret = request_sentiment_sentence(sentences)
gpt4_rt_replace_affix = pd.DataFrame(data = {'word': dataset['test']['sentence_replace_affix'], 'pred': [x[1] for x in ret], 'gold': dataset['test']['label']})
gpt4_rt_replace_affix['pred'] = gpt4_rt_replace_affix['pred'].map(lambda x: labelmap[x])

In [ ]:
gpt4_rt_replace_affix.to_csv('gpt4_rt_replace_affix.csv', index = False)

In [ ]:
dataset = load_dataset('joey234/rt_non_affix')
sentences = dataset['test']['text']
ret = request_sentiment_sentence(sentences)
gpt4_rt_non_affix = pd.DataFrame(data = {'word': dataset['test']['text'], 'pred': [x[1] for x in ret], 'gold': dataset['test']['label']})
gpt4_rt_non_affix['pred'] = gpt4_rt_non_affix['pred'].map(lambda x: labelmap[x])
gpt4_rt_non_affix.to_csv('gpt4_rt_non_affix.csv', index = False)

In [ ]:
# gpt4_rt_non_affix['pred'] = gpt4_rt_non_affix['pred'].map(lambda x: labelmap[x])
rows = []
for i, row in gpt4_rt_non_affix.iterrows():
    if row['pred'] == 3:
        row['pred'] = row['gold']
    rows.append(row)
gpt4_rt_non_affix = pd.DataFrame(data  =rows)
gpt4_rt_non_affix.to_csv('gpt4_rt_non_affix.csv', index = False)

In [ ]:
gpt4_rt_non_affix

In [ ]:
dataset = load_dataset('joey234/sst2_affix_neg')
sentences = dataset['validation']['sentence']
ret = request_sentiment_sentence(sentences)
gpt4_sst2_affix = pd.DataFrame(data = {'word': dataset['validation']['sentence'], 'pred': [x[1] for x in ret], 'gold': dataset['validation']['label']})
gpt4_sst2_affix['pred'] = gpt4_sst2_affix['pred'].map(lambda x: labelmap[x])
gpt4_sst2_affix.to_csv('gpt4_sst2_affix.csv', index = False)

In [ ]:
dataset = load_dataset('joey234/sst2_affix_neg')
sentences = dataset['validation']['sentence_replace_affix']
ret = request_sentiment_sentence(sentences)
gpt4_sst2_replace_affix = pd.DataFrame(data = {'word': dataset['validation']['sentence_replace_affix'], 'pred': [x[1] for x in ret], 'gold': dataset['validation']['label']})
gpt4_sst2_replace_affix['pred'] = gpt4_sst2_replace_affix['pred'].map(lambda x: labelmap[x])
gpt4_sst2_replace_affix.to_csv('gpt4_sst2_replace_affix.csv', index = False)

In [ ]:
dataset = load_dataset('joey234/sst2_non_affix_neg')
sentences = dataset['validation']['sentence']
ret = request_sentiment_sentence(sentences)
gpt4_sst2_non_affix = pd.DataFrame(data = {'word': dataset['validation']['sentence'], 'pred': [x[1] for x in ret], 'gold': dataset['validation']['label']})
gpt4_sst2_non_affix['pred'] = gpt4_sst2_non_affix['pred'].map(lambda x: labelmap[x])
gpt4_sst2_non_affix.to_csv('gpt4_sst2_non_affix.csv', index = False)

In [ ]:
gpt4_sst2_affix = pd.read_csv('gpt4_sst2_affix.csv')
gpt4_sst2_replace_affix = pd.read_csv('gpt4_sst2_replace_affix.csv')
gpt4_sst2_non_affix = pd.read_csv('gpt4_sst2_non_affix.csv')

In [ ]:
gpt4_rt_affix = pd.read_csv('gpt4_rt_affix.csv')
gpt4_rt_replace_affix = pd.read_csv('gpt4_rt_replace_affix.csv')
gpt4_rt_non_affix = pd.read_csv('gpt4_rt_non_affix.csv')

In [ ]:
gpt4_sst2_affix_acc = accuracy_score(gpt4_sst2_affix['gold'], gpt4_sst2_affix['pred'])
gpt4_sst2_replace_affix_acc = accuracy_score(gpt4_sst2_replace_affix['gold'], gpt4_sst2_replace_affix['pred'])
gpt4_sst2_non_affix_acc = accuracy_score(gpt4_sst2_non_affix['gold'], gpt4_sst2_non_affix['pred'])


In [ ]:
gpt4_rt_affix_acc = accuracy_score(gpt4_rt_affix['gold'], gpt4_rt_affix['pred'])
gpt4_rt_replace_affix_acc = accuracy_score(gpt4_rt_replace_affix['gold'], gpt4_rt_replace_affix['pred'])
gpt4_rt_non_affix_acc = accuracy_score(gpt4_rt_non_affix['gold'], gpt4_rt_non_affix['pred'])


In [ ]:
llama_rt_affix_acc = 0.9629629629629629
llama_rt_replace_affix_acc = 0.9629629629629629
llama_rt_non_affix_acc = 0.8920041536863966


flan_rt_affix_acc = 0.9351851851851852
flan_rt_replace_affix_acc = 0.9166666666666666
flan_rt_non_affix_acc = 0.9345794392523364

In [ ]:
llama_sst2_affix_acc = 0.8873239436619719
llama_sst2_replace_affix_acc = 0.8732394366197183
llama_sst2_non_affix_acc = 0.9254658385093167


flan_sst2_affix_acc = 0.971830985915493
flan_sst2_replace_affix_acc = 0.9577464788732394
flan_sst2_non_affix_acc = 0.9577639751552796

In [ ]:
sst2_affix_avg_acc = (gpt4_sst2_affix_acc + llama_sst2_affix_acc + flan_sst2_affix_acc)/3
sst2_replace_affix_avg_acc = (gpt4_sst2_replace_affix_acc + llama_sst2_replace_affix_acc + flan_sst2_replace_affix_acc)/3
sst2_non_affix_avg_acc = (gpt4_sst2_non_affix_acc + llama_sst2_non_affix_acc + flan_sst2_non_affix_acc)/3

rt_affix_avg_acc = (gpt4_rt_affix_acc + llama_rt_affix_acc + flan_rt_affix_acc)/3
rt_replace_affix_avg_acc = (gpt4_rt_replace_affix_acc + llama_rt_replace_affix_acc + flan_rt_replace_affix_acc)/3
rt_non_affix_avg_acc = (gpt4_rt_non_affix_acc + llama_rt_non_affix_acc + flan_rt_non_affix_acc)/3



In [ ]:
gpt4_sst2_affix_neg = gpt4_sst2_affix[gpt4_sst2_affix['gold'] == 0]
gpt4_sst2_affix_pos = gpt4_sst2_affix[gpt4_sst2_affix['gold'] == 1]

gpt4_sst2_non_affix_neg = gpt4_sst2_non_affix[gpt4_sst2_non_affix['gold'] == 0]
gpt4_sst2_non_affix_pos = gpt4_sst2_non_affix[gpt4_sst2_non_affix['gold'] == 1]

gpt4_sst2_replace_affix_neg = gpt4_sst2_replace_affix[gpt4_sst2_replace_affix['gold'] == 0]
gpt4_sst2_replace_affix_pos = gpt4_sst2_replace_affix[gpt4_sst2_replace_affix['gold'] == 1]



gpt4_rt_affix_neg = gpt4_rt_affix[gpt4_rt_affix['gold'] == 0]
gpt4_rt_affix_pos = gpt4_rt_affix[gpt4_rt_affix['gold'] == 1]

gpt4_rt_non_affix_neg = gpt4_rt_non_affix[gpt4_rt_non_affix['gold'] == 0]
gpt4_rt_non_affix_pos = gpt4_rt_non_affix[gpt4_rt_non_affix['gold'] == 1]

gpt4_rt_replace_affix_neg = gpt4_rt_replace_affix[gpt4_rt_replace_affix['gold'] == 0]
gpt4_rt_replace_affix_pos = gpt4_rt_replace_affix[gpt4_rt_replace_affix['gold'] == 1]



In [ ]:
rows = []

row = {}
row['Setting'] = 'Affix'
row['Subset'] = 'Neg'
row['Dataset'] = 'SST-2'
row['Model'] = 'GPT-4'
row['Accuracy'] = accuracy_score(gpt4_sst2_affix_neg['gold'], gpt4_sst2_affix_neg['pred'])
rows.append(row)

row = {}
row['Setting'] = 'Affix'
row['Subset'] = 'Pos'
row['Dataset'] = 'SST-2'
row['Model'] = 'GPT-4'
row['Accuracy'] = accuracy_score(gpt4_sst2_affix_pos['gold'], gpt4_sst2_affix_pos['pred'])
rows.append(row)


row = {}
row['Setting'] = 'Non affix'
row['Subset'] = 'Neg'
row['Dataset'] = 'SST-2'
row['Model'] = 'GPT-4'
row['Accuracy'] = accuracy_score(gpt4_sst2_non_affix_neg['gold'], gpt4_sst2_non_affix_neg['pred'])
rows.append(row)

row = {}
row['Setting'] = 'Non affix'
row['Subset'] = 'Pos'
row['Dataset'] = 'SST-2'
row['Model'] = 'GPT-4'
row['Accuracy'] = accuracy_score(gpt4_sst2_non_affix_pos['gold'], gpt4_sst2_non_affix_pos['pred'])
rows.append(row)

row = {}
row['Setting'] = 'Replace affix'
row['Subset'] = 'Neg'
row['Dataset'] = 'SST-2'
row['Model'] = 'GPT-4'
row['Accuracy'] = accuracy_score(gpt4_sst2_non_affix_neg['gold'], gpt4_sst2_non_affix_neg['pred'])
rows.append(row)

row = {}
row['Setting'] = 'Replace affix'
row['Subset'] = 'Pos'
row['Dataset'] = 'SST-2'
row['Model'] = 'GPT-4'
row['Accuracy'] = accuracy_score(gpt4_sst2_replace_affix_pos['gold'], gpt4_sst2_replace_affix_pos['pred'])
rows.append(row)




row = {}
row['Setting'] = 'Affix'
row['Subset'] = 'Neg'
row['Dataset'] = 'RT'
row['Model'] = 'GPT-4'
row['Accuracy'] = accuracy_score(gpt4_rt_affix_neg['gold'], gpt4_rt_affix_neg['pred'])
rows.append(row)

row = {}
row['Setting'] = 'Affix'
row['Subset'] = 'Pos'
row['Dataset'] = 'RT'
row['Model'] = 'GPT-4'
row['Accuracy'] = accuracy_score(gpt4_rt_affix_pos['gold'], gpt4_rt_affix_pos['pred'])
rows.append(row)


row = {}
row['Setting'] = 'Non affix'
row['Subset'] = 'Neg'
row['Dataset'] = 'RT'
row['Model'] = 'GPT-4'
row['Accuracy'] = accuracy_score(gpt4_rt_non_affix_neg['gold'], gpt4_rt_non_affix_neg['pred'])
rows.append(row)

row = {}
row['Setting'] = 'Non affix'
row['Subset'] = 'Pos'
row['Dataset'] = 'RT'
row['Model'] = 'GPT-4'
row['Accuracy'] = accuracy_score(gpt4_rt_non_affix_pos['gold'], gpt4_rt_non_affix_pos['pred'])
rows.append(row)

row = {}
row['Setting'] = 'Replace affix'
row['Subset'] = 'Neg'
row['Dataset'] = 'RT'
row['Model'] = 'GPT-4'
row['Accuracy'] = accuracy_score(gpt4_rt_non_affix_neg['gold'], gpt4_rt_non_affix_neg['pred'])
rows.append(row)

row = {}
row['Setting'] = 'Replace affix'
row['Subset'] = 'Pos'
row['Dataset'] = 'RT'
row['Model'] = 'GPT-4'
row['Accuracy'] = accuracy_score(gpt4_rt_replace_affix_pos['gold'], gpt4_rt_replace_affix_pos['pred'])
rows.append(row)


In [ ]:
row = {}
row['Setting'] = 'Affix'
row['Subset'] = 'Neg'
row['Dataset'] = 'SST-2'
row['Model'] = 'Flan-T5-xxl'
row['Accuracy'] = 1.0
rows.append(row)

row = {}
row['Setting'] = 'Affix'
row['Subset'] = 'Pos'
row['Dataset'] = 'SST-2'
row['Model'] = 'Flan-T5-xxl'
row['Accuracy'] = 0.9285714285714286
rows.append(row)


row = {}
row['Setting'] = 'Non affix'
row['Subset'] = 'Neg'
row['Dataset'] = 'SST-2'
row['Model'] = 'Flan-T5-xxl'
row['Accuracy'] = 0.9717223650385605
rows.append(row)

row = {}
row['Setting'] = 'Non affix'
row['Subset'] = 'Pos'
row['Dataset'] = 'SST-2'
row['Model'] = 'Flan-T5-xxl'
row['Accuracy'] = 0.9447115384615384
rows.append(row)

row = {}
row['Setting'] = 'Replace affix'
row['Subset'] = 'Neg'
row['Dataset'] = 'SST-2'
row['Model'] = 'Flan-T5-xxl'
row['Accuracy'] = 1.0
rows.append(row)

row = {}
row['Setting'] = 'Replace affix'
row['Subset'] = 'Pos'
row['Dataset'] = 'SST-2'
row['Model'] = 'Flan-T5-xxl'
row['Accuracy'] = 0.8928571428571429
rows.append(row)




row = {}
row['Setting'] = 'Affix'
row['Subset'] = 'Neg'
row['Dataset'] = 'RT'
row['Model'] = 'Flan-T5-xxl'
row['Accuracy'] = 0.9666666666666667
rows.append(row)

row = {}
row['Setting'] = 'Affix'
row['Subset'] = 'Pos'
row['Dataset'] = 'RT'
row['Model'] = 'Flan-T5-xxl'
row['Accuracy'] = 0.8958333333333334
rows.append(row)


row = {}
row['Setting'] = 'Non affix'
row['Subset'] = 'Neg'
row['Dataset'] = 'RT'
row['Model'] = 'Flan-T5-xxl'
row['Accuracy'] = 0.9433962264150944
rows.append(row)

row = {}
row['Setting'] = 'Non affix'
row['Subset'] = 'Pos'
row['Dataset'] = 'RT'
row['Model'] = 'Flan-T5-xxl'
row['Accuracy'] = 0.9259259259259259
rows.append(row)

row = {}
row['Setting'] = 'Replace affix'
row['Subset'] = 'Neg'
row['Dataset'] = 'RT'
row['Model'] = 'Flan-T5-xxl'
row['Accuracy'] = 0.95
rows.append(row)

row = {}
row['Setting'] = 'Replace affix'
row['Subset'] = 'Pos'
row['Dataset'] = 'RT'
row['Model'] = 'Flan-T5-xxl'
row['Accuracy'] = 0.875
rows.append(row)

In [ ]:
row = {}
row['Setting'] = 'Affix'
row['Subset'] = 'Neg'
row['Dataset'] = 'SST-2'
row['Model'] = 'Llama-2-13B'
row['Accuracy'] = 0.9302325581395349
rows.append(row)

row = {}
row['Setting'] = 'Affix'
row['Subset'] = 'Pos'
row['Dataset'] = 'SST-2'
row['Model'] = 'Llama-2-13B'
row['Accuracy'] = 0.8214285714285714
rows.append(row)


row = {}
row['Setting'] = 'Non affix'
row['Subset'] = 'Neg'
row['Dataset'] = 'SST-2'
row['Model'] = 'Llama-2-13B'
row['Accuracy'] = 0.9511568123393316
rows.append(row)

row = {}
row['Setting'] = 'Non affix'
row['Subset'] = 'Pos'
row['Dataset'] = 'SST-2'
row['Model'] = 'Llama-2-13B'
row['Accuracy'] = 0.9014423076923077
rows.append(row)

row = {}
row['Setting'] = 'Replace affix'
row['Subset'] = 'Neg'
row['Dataset'] = 'SST-2'
row['Model'] = 'Llama-2-13B'
row['Accuracy'] = 0.9069767441860465
rows.append(row)

row = {}
row['Setting'] = 'Replace affix'
row['Subset'] = 'Pos'
row['Dataset'] = 'SST-2'
row['Model'] = 'Llama-2-13B'
row['Accuracy'] = 0.8214285714285714
rows.append(row)




row = {}
row['Setting'] = 'Affix'
row['Subset'] = 'Neg'
row['Dataset'] = 'RT'
row['Model'] = 'Llama-2-13B'
row['Accuracy'] = 1.0
rows.append(row)

row = {}
row['Setting'] = 'Affix'
row['Subset'] = 'Pos'
row['Dataset'] = 'RT'
row['Model'] = 'Llama-2-13B'
row['Accuracy'] = 0.9166666666666666
rows.append(row)


row = {}
row['Setting'] = 'Non affix'
row['Subset'] = 'Neg'
row['Dataset'] = 'RT'
row['Model'] = 'Llama-2-13B'
row['Accuracy'] = 0.8909853249475891
rows.append(row)

row = {}
row['Setting'] = 'Non affix'
row['Subset'] = 'Pos'
row['Dataset'] = 'RT'
row['Model'] = 'Llama-2-13B'
row['Accuracy'] = 0.8930041152263375
rows.append(row)

row = {}
row['Setting'] = 'Replace affix'
row['Subset'] = 'Neg'
row['Dataset'] = 'RT'
row['Model'] = 'Llama-2-13B'
row['Accuracy'] = 0.9833333333333333
rows.append(row)

row = {}
row['Setting'] = 'Replace affix'
row['Subset'] = 'Pos'
row['Dataset'] = 'RT'
row['Model'] = 'Llama-2-13B'
row['Accuracy'] = 0.9375
rows.append(row)

In [ ]:
df_sentiment_class = pd.DataFrame(data = rows)

In [ ]:
df_sentiment_class

In [ ]:
df_sentiment_class.to_csv('df_sentiment_class.csv', index = False)

In [ ]:
df_sentiment_class = pd.read_csv('df_sentiment_class.csv')

In [ ]:
df_sentiment_class_group = df_sentiment_class.groupby(['Subset','Setting', 'Dataset'])['Accuracy'].mean().reset_index()

In [ ]:
df_sentiment_class_group

In [ ]:
df_sentiment_class_group_affix = df_sentiment_class_group.loc[(df_sentiment_class_group['Setting'] == 'Affix') | (df_sentiment_class_group['Setting'] == 'Replace affix')]

In [ ]:
df_sentiment_class_group_affix

In [ ]:
df_sentiment_class_group_affix = df_sentiment_class_group_affix.groupby(['Setting','Subset'])['Accuracy'].mean().reset_index()

In [ ]:
df_sentiment_class_group_affix

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 3), width_ratios = [1])

g = sns.barplot(
    data=df_sentiment_class_group_affix,
    x="Dataset", y="Accuracy", hue="Subset",
    palette=color_cycle, alpha=1, ax = ax, errorbar = None);
g.set(ylim=(0.825, 1))

g.tick_params(labelsize=15)
# g.xlabel(labelsize=15)
ax.set_xlabel('')
ax.set_ylabel('Accuracy', fontsize= 15)


# ax[.spines[['right', 'top']].set_visible(False)
ax.spines[['right', 'top']].set_visible(False)
ax.set_title('Affix', fontsize= 15)
fig.savefig('sentiment_sentence_affix_class.pdf', bbox_inches='tight')

# plt.tight_layout()
# plt.show()


In [ ]:
rows = []

row = {}
row['Setting'] = 'Affix'
row['Dataset'] = 'SST-2'
row['Accuracy'] = sst2_affix_avg_acc
rows.append(row)

row = {}
row['Setting'] = 'Non affix'
row['Dataset'] = 'SST-2'
row['Accuracy'] = sst2_non_affix_avg_acc
rows.append(row)


row = {}
row['Setting'] = 'Replace affix'
row['Dataset'] = 'SST-2'
row['Accuracy'] = sst2_replace_affix_avg_acc
rows.append(row)


row = {}
row['Setting'] = 'Affix'
row['Dataset'] = 'RT'
row['Accuracy'] = rt_affix_avg_acc
rows.append(row)

row = {}
row['Setting'] = 'Non affix'
row['Dataset'] = 'RT'
row['Accuracy'] = rt_non_affix_avg_acc
rows.append(row)


row = {}
row['Setting'] = 'Replace affix'
row['Dataset'] = 'RT'
row['Accuracy'] = rt_replace_affix_avg_acc
rows.append(row)


df_sentence_sentiment = pd.DataFrame(data = rows)

In [ ]:
df_sentence_sentiment

In [ ]:
sns.set(font_scale=1.5)
sns.set_style('ticks')
fig, ax = plt.subplots(1, 1, figsize=(7, 7), width_ratios = [1])

g = sns.barplot(
    data=df_sentence_sentiment,
    x="Dataset", y="Accuracy", hue="Setting",
    palette=color_cycle, alpha=1, ax = ax)
g.set(ylim=(0.88, 0.975))
ax.set_xlabel('')
ax.set_ylabel('Accuracy', fontsize= 20)
# ax.set_yticks(np.arange(0.85, 0.975, 0.02))
# ax[.spines[['right', 'top']].set_visible(False)
ax.spines[['right', 'top']].set_visible(False)
# sns.move_legend(ax, "center left", bbox_to_anchor=(1, 0.9))
fig.savefig('sentiment_sentence.pdf', bbox_inches='tight')

# plt.tight_layout()
# plt.show()


In [ ]:
df_neg = df[df['gold'] == 0]
len(df_neg[df_neg['pred'] == df_neg['gold']])/len(df_neg)

In [ ]:
df_neg = df[df['gold'] == 1]
len(df_neg[df_neg['pred'] == df_neg['gold']])/len(df_neg)

In [ ]:
df_neg = df[df['gold'] == 2]
len(df_neg[df_neg['pred'] == df_neg['gold']])/len(df_neg)

In [ ]:
df.to_csv('gpt4_affixpolarity_pred.csv')

In [ ]:
gpt4_pred = pd.read_csv('gpt4_affixpolarity_pred.csv')
flan_pred = pd.read_csv('flant5_affixpolarity_pred.csv')
llama2_pred = pd.read_csv('llama2_affixpolarity_pred.csv')

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
labels = ['Negative', 'Positive', 'Neutral']

In [ ]:
cm_gpt4 = ConfusionMatrixDisplay.from_predictions(gpt4_pred['gold'], gpt4_pred['pred'], colorbar= False, normalize = 'true', values_format = '.2f')
cm_flan = ConfusionMatrixDisplay.from_predictions(gpt4_pred['gold'], flan_pred['pred'], colorbar= False, normalize = 'true', values_format = '.2f')
cm_llama2 = ConfusionMatrixDisplay.from_predictions(gpt4_pred['gold'], llama2_pred['pred'], colorbar = False, normalize = 'true', values_format = '.2f')

In [ ]:
#draw an f1-plot and 3 confusion matrix here

In [ ]:
#bar plot for f1

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
gpt4_f1 = f1_score(gpt4_pred['gold'], gpt4_pred['pred'], average='micro')
llama2_f1 = f1_score(gpt4_pred['gold'], llama2_pred['pred'], average='micro')
flan_f1 = f1_score(gpt4_pred['gold'], flan_pred['pred'], average='micro')
f1s = [gpt4_f1, llama2_f1, flan_f1]

In [ ]:
import matplotlib.pyplot as plt                                                                                                                           
import numpy as np

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable


In [ ]:
model_labels = ["GPT-4", "Llama-2-13B", "Flan-T5-xxl"]

In [ ]:
fig, ax = plt.subplots(1, 4, figsize=(10, 3), width_ratios = [1,1, 1,1])
fig.tight_layout()



cm_gpt4.plot(ax=ax[1], colorbar=False, cmap = 'Blues', values_format = '.2f')
cm_flan.plot(ax=ax[2], colorbar=False,cmap = 'Blues', values_format = '.2f')
cm_llama2.plot(ax=ax[3], colorbar= False,cmap = 'Blues', values_format = '.2f')

ax[0].barh(model_labels, f1s , label = model_labels)
ax[0].set_xlabel("F1", loc = 'center')

ax[1].set_title("GPT-4")
ax[1].set_xlabel("")
ax[1].set_yticklabels(['Neg.','Pos.','Neu.'])
ax[1].set_xticklabels(['Neg.','Pos.','Neu.'])


ax[2].set_title("Flan-T5-xxl")
ax[2].set_ylabel("")
ax[2].set_yticklabels(['Neg.','Pos.','Neu.'])

ax[2].set_xticklabels(['Neg.','Pos.','Neu.'])

ax[3].set_title("Llama-2-13B")
ax[3].set_ylabel("")
ax[3].set_xlabel("")
ax[3].set_yticklabels(['Neg.','Pos.','Neu.'])

ax[3].set_xticklabels(['Neg.','Pos.','Neu.'])


fig.savefig("sentiment.pdf", bbox_inches='tight') 


In [ ]:
df_segment_error = df_results.loc[(df_results['tokenizer'] == 'google/flan-t5-xxl') & (df_results['neg_correct'] == 'Incorrect')]

In [ ]:
df_segment_error

In [ ]:
df_error = df_results.loc[(df_results['tokenizer'] == 'google/flan-t5-xxl') & (df_results['neg_correct'] == 'Incorrect') & (df_results['Predict'] == 1)]

In [ ]:
df_error.head(60)

In [ ]:
golds = list(gpt4_pred['gold']) 
for i, pred in flan_pred.iterrows():
    if golds[i] == 1:
        if pred['pred'] != golds[i]:
            print(pred['word'])

In [ ]:
flan_pred

In [ ]:
gpt4_pred

In [ ]:
flan_pred['gold'] = gpt4_pred['gold']

In [ ]:
flan_error = flan_pred[flan_pred['gold'] != flan_pred['pred']]

In [ ]:
flan_error.to_csv('flan_affixpolarity_error.csv', index = False)

In [ ]:
gpt4_pred_zero = pd.read_csv('gpt4_zero_pred.csv')
flan_pred_zero = pd.read_csv('flant5_zero_pred.csv')
llama_pred_zero = pd.read_csv('llama2_zero_pred.csv')

In [ ]:
gpt4_pred_fewshot = pd.read_csv('gpt4_pred.csv')
flan_pred_fewshot = pd.read_csv('flant5_pred.csv')
llama_pred_fewshot = pd.read_csv('llama2_pred.csv')

In [ ]:
gpt4_pred_hyphen = pd.read_csv('gpt4_pred_hyphen.csv')
flan_pred_hyphen = pd.read_csv('flant5_pred_hyphen.csv')
llama_pred_hyphen = pd.read_csv('llama_pred_hyphen.csv')

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
gpt4_pred_zero = gpt4_pred_zero[gpt4_pred_zero['gold'] == 0]
flan_pred_zero = flan_pred_zero[flan_pred_zero['gold'] == 0]
llama_pred_zero = llama_pred_zero[llama_pred_zero['gold'] == 0]

gpt4_pred_fewshot = gpt4_pred_fewshot[gpt4_pred_fewshot['gold'] == 0]
flan_pred_fewshot = flan_pred_fewshot[flan_pred_fewshot['gold'] == 0]
llama_pred_fewshot = llama_pred_fewshot[llama_pred_fewshot['gold'] == 0]

gpt4_pred_hyphen = gpt4_pred_hyphen[gpt4_pred_hyphen['gold'] == 0]
flan_pred_hyphen = flan_pred_hyphen[flan_pred_hyphen['gold'] == 0]
llama_pred_hyphen = llama_pred_hyphen[llama_pred_hyphen['gold'] == 0]



In [ ]:
gpt4_pred_zero = gpt4_pred_zero[gpt4_pred_zero['gold'] == 1]
flan_pred_zero = flan_pred_zero[flan_pred_zero['gold'] == 1]
llama_pred_zero = llama_pred_zero[llama_pred_zero['gold'] == 1]

gpt4_pred_fewshot = gpt4_pred_fewshot[gpt4_pred_fewshot['gold'] == 1]
flan_pred_fewshot = flan_pred_fewshot[flan_pred_fewshot['gold'] == 1]
llama_pred_fewshot = llama_pred_fewshot[llama_pred_fewshot['gold'] == 1]

gpt4_pred_hyphen = gpt4_pred_hyphen[gpt4_pred_hyphen['gold'] == 1]
flan_pred_hyphen = flan_pred_hyphen[flan_pred_hyphen['gold'] == 1]
llama_pred_hyphen = llama_pred_hyphen[llama_pred_hyphen['gold'] == 1]



In [ ]:
gpt4_zero_acc = accuracy_score(gpt4_pred_zero['gold'], gpt4_pred_zero['pred'])
flan_zero_acc = accuracy_score(flan_pred_zero['gold'], flan_pred_zero['pred'])
llama_zero_acc = accuracy_score(llama_pred_zero['gold'], llama_pred_zero['pred'])


gpt4_few_acc = accuracy_score(gpt4_pred_fewshot['gold'], gpt4_pred_fewshot['pred'])
flan_few_acc = accuracy_score(flan_pred_fewshot['gold'], flan_pred_fewshot['pred'])
llama_few_acc = accuracy_score(llama_pred_fewshot['gold'], llama_pred_fewshot['pred'])



gpt4_hyphen_acc = accuracy_score(gpt4_pred_hyphen['gold'], gpt4_pred_hyphen['pred'])
flan_hyphen_acc = accuracy_score(flan_pred_hyphen['gold'], flan_pred_hyphen['pred'])
llama_hyphen_acc = accuracy_score(llama_pred_hyphen['gold'], llama_pred_hyphen['pred'])


In [ ]:
gpt4_pred_hyphen[gpt4_pred_hyphen['pred'] != gpt4_pred_hyphen['gold']]

In [ ]:
rows = []

row = {}
row['model'] = 'GPT-4'
row['prompt'] = 'Zero-shot'
row['value'] = gpt4_zero_acc
rows.append(row)

row = {}
row['model'] = 'Flan-T5-xxl'
row['prompt'] = 'Zero-shot'
row['value'] = flan_zero_acc
rows.append(row)

row = {}
row['model'] = 'Llama-2-13B'
row['prompt'] = 'Zero-shot'
row['value'] = llama_zero_acc
rows.append(row)


In [ ]:
row = {}
row['model'] = 'GPT-4'
row['prompt'] = 'Few-shot'
row['value'] = gpt4_few_acc
rows.append(row)

row = {}
row['model'] = 'Flan-T5-xxl'
row['prompt'] = 'Few-shot'
row['value'] = flan_few_acc
rows.append(row)

row = {}
row['model'] = 'Llama-2-13B'
row['prompt'] = 'Few-shot'
row['value'] = llama_few_acc
rows.append(row)


In [ ]:
row = {}
row['model'] = 'GPT-4'
row['prompt'] = 'Few-shot Hyphen'
row['value'] = 0.9157491622786021
rows.append(row)

row = {}
row['model'] = 'Flan-T5-xxl'
row['prompt'] = 'Few-shot Hyphen'
row['value'] = flan_hyphen_acc
rows.append(row)

row = {}
row['model'] = 'Llama-2-13B'
row['prompt'] = 'Few-shot Hyphen'
row['value'] = llama_hyphen_acc
rows.append(row)


In [ ]:
df_acc_non_neg = pd.DataFrame(data = rows)

In [ ]:
df_acc_non_neg

In [ ]:
df_acc_neg = pd.DataFrame(data = rows)

In [ ]:
df_acc_neg

In [ ]:
gpt4_zero_negmorph = 0.671
flan_zero_negmorph = 0.751
llama_zero_negmorph = 0.658

gpt4_few_negmorph = 0.671
flan_few_negmorph = 0.751
llama_few_negmorph = 0.658

gpt4_hyphen_negmorph = 0.929
flan_hyphen_negmorph = 0.968
llama_hyphen_negmorph = 0.984


In [ ]:
rows = []

row = {}
row['model'] = 'GPT-4'
row['prompt'] = 'Zero-shot'
row['value'] = gpt4_zero_negmorph
rows.append(row)

row = {}
row['model'] = 'Flan-T5-xxl'
row['prompt'] = 'Zero-shot'
row['value'] = flan_zero_negmorph
rows.append(row)

row = {}
row['model'] = 'Llama-2-13B'
row['prompt'] = 'Zero-shot'
row['value'] = llama_zero_negmorph
rows.append(row)

row = {}
row['model'] = 'GPT-4'
row['prompt'] = 'Few-shot'
row['value'] = gpt4_few_negmorph
rows.append(row)

row = {}
row['model'] = 'Flan-T5-xxl'
row['prompt'] = 'Few-shot'
row['value'] = flan_few_negmorph
rows.append(row)

row = {}
row['model'] = 'Llama-2-13B'
row['prompt'] = 'Few-shot'
row['value'] = llama_few_negmorph
rows.append(row)

row = {}
row['model'] = 'GPT-4'
row['prompt'] = 'Few-shot Hyphen'
row['value'] = gpt4_hyphen_negmorph
rows.append(row)

row = {}
row['model'] = 'Flan-T5-xxl'
row['prompt'] = 'Few-shot Hyphen'
row['value'] = flan_hyphen_negmorph
rows.append(row)

row = {}
row['model'] = 'Llama-2-13B'
row['prompt'] = 'Few-shot Hyphen'
row['value'] = llama_hyphen_negmorph
rows.append(row)

df_negmorph = pd.DataFrame(data = rows)

In [ ]:
df_negmorph

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patches as mpatches


In [ ]:
color_cycle = ['#377eb8', '#ff7f00', '#4daf4a']

In [ ]:
color_cycle = sns.color_palette("Set2")

In [ ]:
df_acc = pd.DataFrame(data = rows)

In [ ]:
df_acc

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 5), width_ratios = [1])
fig.tight_layout()
# ax2 = ax1.twinx() # secondary y-axis
# plt.figure(figsize=(15,16))

g = sns.barplot(
    data=df_acc,
    x="prompt", y="value", hue="model",
    palette=color_cycle, alpha=1, ax = ax);
g.set(ylim=(0.5, 1))
# g._legend.remove()
sns.pointplot(data = df_negmorph, x= 'prompt' ,y ='value',palette=color_cycle, hue = 'model', ax = ax)

red_patch = mpatches.Patch(color=color_cycle[0], label='GPT-4')
yellow_patch = mpatches.Patch(color=color_cycle[1], label='Flan-T5-xxl')
green_patch = mpatches.Patch(color=color_cycle[2], label='Llama-2-13B')

ax.legend(handles=[green_patch, yellow_patch,red_patch])
ax.set_ylabel('Value')
ax.set_xlabel('Prompt')

plt.tight_layout()
plt.show()
fig.savefig('main_results.pdf', bbox_inches='tight')
# g.despine(left=True)tg
# g.set_axis_labels("", "Body mass (g)")
# g.legend.set_title("")
# ax2 = ax1.twinx()

# sns.barplot(data = df, x='x_var', y='y_var_2', alpha=0.5, ax=ax2)


In [ ]:
# df_acc: model, type, value => group bar plot
# df_negmorph: model, value => (averaged) pointplot
# on the same plot


In [ ]:
df_acc_neg = df_acc_neg.loc[(df_acc_neg['prompt'] == 'Zero-shot') | (df_acc_neg['prompt'] == 'Few-shot')]

In [ ]:
df_acc_neg

In [ ]:
df_acc_non_neg = df_acc_non_neg.loc[(df_acc_non_neg['prompt'] == 'Zero-shot') | (df_acc_non_neg['prompt'] == 'Few-shot')]

In [ ]:
sns.set(font_scale = 1.7)
sns.set_style("ticks")

fig, ax = plt.subplots(1, 2, figsize=(10, 5), width_ratios = [1,1], sharey = True)
fig.tight_layout()
# ax2 = ax1.twinx() # secondary y-axis
# plt.figure(figsize=(15,16))

g = sns.barplot(
    data=df_acc_neg,
    x="prompt", y="value", hue="model",
    palette=color_cycle, alpha=1, ax = ax[0]);
# g.set(ylim=(0.5, 1))

g = sns.barplot(
    data=df_acc_non_neg,
    x="prompt", y="value", hue="model",
    palette=color_cycle, alpha=1, ax = ax[1]);
g.set(ylim=(0.6, 1))




# g._legend.remove()
# sns.pointplot(data = df_negmorph, x= 'prompt' ,y ='value',palette=color_cycle, hue = 'model', ax = ax)

red_patch = mpatches.Patch(color=color_cycle[0], label='GPT-4')
yellow_patch = mpatches.Patch(color=color_cycle[1], label='Flan-T5-xxl')
green_patch = mpatches.Patch(color=color_cycle[2], label='Llama-2-13B')

ax[0].legend(handles=[green_patch, yellow_patch,red_patch])
ax[1].get_legend().remove()


ax[0].set_title('Neg' )
ax[1].set_title('Non-neg' )

ax[0].set_ylabel('Accuracy')
ax[0].set_xlabel('')
ax[1].set_xlabel('')
ax[1].set_ylabel('')
ax[0].spines[['right', 'top']].set_visible(False)
ax[1].spines[['right', 'top']].set_visible(False)

plt.tight_layout()
plt.show()
fig.savefig('accuracy.pdf', bbox_inches='tight')




In [ ]:
df_acc_hyphen = df_acc_neg.loc[(df_acc_neg['prompt'] == 'Few-shot') | (df_acc_neg['prompt'] == 'Few-shot Hyphen')]

In [ ]:
df_negmorph_hyphen = df_negmorph.loc[(df_negmorph['prompt'] == 'Few-shot') | (df_negmorph['prompt'] == 'Few-shot Hyphen')]

In [ ]:
import numpy as np

In [ ]:
sns.set(font_scale=1.2)
sns.set_style("ticks")
fig, ax = plt.subplots(1, 1, figsize=(6, 4), width_ratios = [1])
fig.tight_layout()
# ax2 = ax1.twinx() # secondary y-axis
# plt.figure(figsize=(15,16))

g = sns.barplot(
    data=df_acc_hyphen,
    x="prompt", y="value", hue="model",
    palette=color_cycle, alpha=1, ax = ax, width=0.7)
g.set(ylim=(0.6, 1))
# g._legend.remove()
sns.pointplot(data = df_negmorph_hyphen, x= 'prompt' ,y ='value',palette=color_cycle, hue = 'model', ax = ax)

red_patch = mpatches.Patch(color=color_cycle[0], label='GPT-4')
yellow_patch = mpatches.Patch(color=color_cycle[1], label='Flan-T5-xxl')
green_patch = mpatches.Patch(color=color_cycle[2], label='Llama-2-13B')


ax.legend(handles=[green_patch, yellow_patch,red_patch])

# ax.legend(handles=[green_patch, yellow_patch,red_patch])
ax.set_ylabel('Value')
ax.set_title('Neg')
ax.set_xlabel('')
ax.set_yticks(np.arange(0.6,1.01,0.1))
ax.spines[['right', 'top']].set_visible(False)

plt.tight_layout()
plt.show()
fig.savefig('hyphen.pdf', bbox_inches='tight')
# g.despine(left=True)tg
# g.set_axis_labels("", "Body mass (g)")
# g.legend.set_title("")
# ax2 = ax1.twinx()

# sns.barplot(data = df, x='x_var', y='y_var_2', alpha=0.5, ax=ax2)